In [74]:
from tqdm import tqdm
import time
import os
import numpy  as np

In [2]:
import torch
cuda_id = 1
device = torch.device("cuda:%s" % cuda_id if torch.cuda.is_available() else "cpu")
device_name = torch.cuda.get_device_name(cuda_id) if torch.cuda.is_available() else "cpu"
print("We are using the device %s - %s" % (device, device_name))

We are using the device cuda:1 - GeForce RTX 2080 Ti


In [3]:
# Best implementation
model_name = "roberta-large"
optimizer_name = 'Adam'
learning_rate = 1e-5
batch_size = 24
epoch = 5

In [4]:
model_params_path = "./Roberta_Best_Model.pth"

In [5]:
import pandas as pd
def get_info_from_csv(texts_path, labels_path):
    texts = pd.read_csv(texts_path, header=0, names=['ID', 'Statement 0', 'Statement 1'])
    labels = pd.read_csv(labels_path, header=None, names=['ID', 'Answer'])['Answer']
    return texts, labels

In [6]:
train_texts, train_labels = get_info_from_csv(
    '../DataSet/Training Data/subtaskA_data_all.csv',
    '../DataSet/Training Data/subtaskA_answers_all.csv'
)

val_texts, val_labels = get_info_from_csv(
    '../DataSet/Dev Data/subtaskA_dev_data.csv',
    '../DataSet/Dev Data/subtaskA_gold_answers.csv'
)

test_texts, test_labels = get_info_from_csv(
    '../DataSet/Test Data/subtaskA_test_data.csv',
    '../DataSet/Test Data/subtaskA_gold_answers.csv'
)

In [ ]:
train_data = pd.concat([train_texts, train_labels], axis=1)
train_data[10]

In [44]:
pro0=train_data.loc[train_data['Answer']==0,['Statement 0']]
pro0.rename(columns=lambda x:x.replace(' 0',''), inplace=True)
pro1=train_data.loc[train_data['Answer']==1,['Statement 1']]
pro1.rename(columns=lambda x:x.replace(' 1',''), inplace=True)
pro=pd.concat([pro0,pro1], axis=0)

Statement
0               He poured orange juice on his cereal.
1                                    He drinks apple.
6                         I like to ride my chocolate
8                        he put elephant into the jug
10                            Eggs eat kis on Easter.
11                             Mark drank a notebook.
12    He went to the police station to withdraw cash.
17  Lisa played jazz music for Moses of the Old Te...
18    the computer broke down and the doctor fixed it
19                                    I drank bleach.

In [50]:
con0=train_data.loc[train_data['Answer']==1,['Statement 0']]
con0.rename(columns=lambda x:x.replace(' 0',''), inplace=True)
con1=train_data.loc[train_data['Answer']==0,['Statement 1']]
con1.rename(columns=lambda x:x.replace(' 1',''), inplace=True)
con=pd.concat([con0,con1], axis=0)

In [51]:
train_data_pro1 = pd.concat([pro,pd.DataFrame([0 for i in range(len(pro))])], axis=1)
train_data_con1 = pd.concat([con,pd.DataFrame([1 for i in range(len(con))])], axis=1)

In [ ]:
train_data_1 = pd.concat([train_data_pro1,train_data_con1], axis=0)

In [ ]:
list(train_data_1[0])

In [65]:
from transformers import BertTokenizerFast, DistilBertTokenizerFast, RobertaTokenizerFast

# Get tokenizer
if model_name.startswith("bert"):
    tokenizer = BertTokenizerFast.from_pretrained(model_name)
elif model_name.startswith("distilbert"):
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
elif model_name.startswith("roberta"):
    tokenizer = RobertaTokenizerFast.from_pretrained('../roberta-large')

# Tokenization for texts
train_encodings = tokenizer(list(train_data_1["Statement"]), truncation=True, padding=True,max_length=50)
#val_encodings = tokenizer(list(val_texts["Statement 0"]), list(val_texts["Statement 1"]), truncation=True, padding=True)
#test_encodings = tokenizer(list(test_texts["Statement 0"]), list(test_texts["Statement 1"]), truncation=True, padding=True)

In [66]:
class ComVEDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [75]:
train_dataset = ComVEDataset(train_encodings, np.array(train_data_1[0]))

In [ ]:
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, RobertaForSequenceClassification, AdamW

# Loading the model
if model_name.startswith("bert"):
    model = BertForSequenceClassification.from_pretrained(model_name)
elif model_name.startswith("distilbert"):
    model = DistilBertForSequenceClassification.from_pretrained(model_name)
elif model_name.startswith("roberta"):
    model = RobertaForSequenceClassification.from_pretrained('../roberta-large')


model.to(device)

# Loading the optimizer
if optimizer_name == "Adam":
    optim = AdamW(model.parameters(), lr=learning_rate)

In [70]:
# Prediction function
def predict(outputs):
    probabilities = torch.softmax(outputs["logits"], dim=1)
    predictions = torch.argmax(probabilities, dim=1)
    return predictions

In [ ]:
# Training
from torch.utils.data import DataLoader

# Set the number of epoch
epoch = epoch

# Get training data by DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Start training
model.train()

train_loss = list()
train_accuracies = list()
highest_accuracy = 0
for epoch_i in range(epoch):
    print('Epoch %s/%s' % (epoch_i + 1, epoch))
    time.sleep(0.3)
    
    correct = 0
    count = 0
    epoch_loss = list()
    
    pbar = tqdm(train_loader)
    for batch in pbar:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs['loss']
        loss.backward()
        optim.step()
        
        # make predictions
        predictions = predict(outputs)

        # count accuracy
        correct += predictions.eq(labels).sum().item()
        count += len(labels)
        accuracy = correct * 1.0 / count

        # show progress along with metrics
        pbar.set_postfix({
            'Loss': '{:.3f}'.format(loss.item()),
            'Accuracy': '{:.3f}'.format(accuracy)
        })
        
        # record the loss for each batch
        epoch_loss.append(loss.item())
        
    pbar.close()
    
    # record the loss and accuracy for each epoch
    train_loss += epoch_loss
    train_accuracies.append(accuracy)
    
    # save the model with the highest accuracy
    if highest_accuracy <= accuracy:
        highest_accuracy = accuracy
    torch.save(model.state_dict(), model_params_path)